In [45]:
import nibabel as nib
from nilearn.connectome import ConnectivityMeasure
import numpy as np
from nilearn import plotting

import pandas as pd
import numpy as np

import os
import matplotlib.pyplot as plt

from scipy.signal import butter, filtfilt, iirfilter, freqs
from scipy.signal import freqz
from scipy import stats
from scipy.stats import pearsonr
from scipy import signal
import os
import subprocess
import time

In [46]:
labels = pd.read_csv("./inputs/labels_reduc.txt", sep=",", header=None)

def savefiles(mtx, title, filename, typ):
    
    if (typ == "png") or (typ == "pngcsv"):
        # save img
        plt.matshow(mtx)
        plt.title(title)
        plt.colorbar()
        plt.clim(0,1)
        plt.xticks(labels[0], labels[1], rotation='vertical', fontsize=7)
        plt.savefig(filename+".png")
        plt.close()

    if (typ == "csv") or (typ == "pngcsv") or (typ == "pngcsv_c"):
        # save csv
        np.savetxt(filename+".csv", mtx , delimiter=",")

    if (typ == "pngcsv_c"):
        # save img
        plt.matshow(mtx)
        plt.title(title)
        plt.colorbar()
        plt.clim(-1,1)
        plt.xticks(labels[0], labels[1], rotation='vertical', fontsize=7)
        plt.savefig(filename+".png")
        plt.close()

In [47]:
# PARAMETERS TO CHANGE ######

group = "CN" # AD or CN

#############################

# Get the list of all files and directories
in_path = "./inputs/"+group+"_proc_roi100/"
dir_list = os.listdir(in_path)

FC_sum = np.zeros([100, 100])
count = 0
S_group_windows = dict()
CS_group = []
for patient in dir_list:

    data_set = pd.read_csv(in_path+patient, header=None)

    # get time series in array
    time_series = np.array(data_set) # (140, 400)
    time_series = time_series[:,:100] # (140, 100)

    # parameters MRI
    N_voxels = len(time_series[1,:]) #100
    N_TR = len(time_series[:,1])# 140 # number of repetition times
    TR = 3 # repetition time in seconds

    # Sample rate and desired cutoff frequencies (in Hz).
    fs = 1/TR
    lowcut = 0.01
    highcut = 0.08

    # filter
    order = 12
    nyq = 0.5*fs
    b, a = butter(order, [lowcut/nyq, highcut/nyq], btype='band')
    w, h = freqz(b, a, fs=fs)


    # remove 1st 3 values value
    remov = 3
    time_series_r = time_series[remov:,:]
    N_TR = N_TR - remov

    # time
    t = np.linspace(0,(TR*N_TR)/60, N_TR-1) # min

    time_series_filt = np.zeros([N_TR, N_voxels])

    # normalization
    for parcel in range(N_voxels):

        time_series_p = (time_series_r[:, parcel]  - np.mean(time_series_r[:, parcel] ))/ np.std(time_series_r[:, parcel])
        time_series_f = filtfilt(b, a, time_series_p)
        time_series_filt[:,parcel] = time_series_f

    tseries_corr = pd.DataFrame(time_series_filt).corr(method ='pearson')
    FC_sum = FC_sum + (tseries_corr + 1)/ 2 # 100, 100

    """Dynamic connectivity analysis"""

    # sliding time–window
    # with width of L = 20 TR slid in steps of 1 TR) approach

    L = 13 # 39s = 13*3 window width
    F = N_TR # 137 time points

    # number of time windows
    W = F - L + 1 # 125 time windows
    
    # calculate dFC over all regions for a finite window time
    start_w = 0
    t_window = []

    S = dict()
    CS = np.zeros([N_voxels, W]) # nodes, graphs

    for window_step in range(W):

        """ S """ # 100, 100
        tseries_wind = pd.DataFrame(time_series_filt[start_w : start_w + L,:])
        R_w = tseries_wind.corr(method ='pearson')
        S_w = (R_w + 1)/ 2 # 100, 100
        S[window_step] = S_w

        """ connectivity strength of each voxel"""

        CS[:,window_step] = S_w.sum(axis=1) # (100, 125)

        start_w = start_w + 1

    # add all windows from a specific patient to a dictionary
    S_group_windows[count] =  S
    CS_group.append(np.mean(CS))

    #savefiles(S_w, "Last Window Simililarity Graph", './outputs/S_last_window/'+group+'/S_'+str(count), "png")
    #savefiles(CS, "Connectivity Strength", './outputs/CS/'+group+'/CS_'+str(count), "png")

    # To estimate how the brain connectivity patterns of different time–
    # windows are associated to each other, a new similarity matrix, Scs(125 × 125)

    # correlation of connectivity strength
    Ccs_df = pd.DataFrame(CS)
    Ccs = Ccs_df.corr(method ='pearson') #(125, 125)
    #Scs = (cs + 1)/ 2
    
    savefiles(Ccs, "Correlation Connectivity Strength", './outputs/Ccs_100_corr/'+group+'/Ccs_'+str(count), "pngcsv_c")
    
    count = count +1

In [49]:
# summary of group stationary FC
FC = FC_sum/count
savefiles(FC, "Functional Connectivity "+group+" Group Average", './outputs/FC_'+group, "pngcsv")


In [50]:
# go to matlab

""" Average S according to Modularity """

# Since modules of Scs(125 × 125) may correspond to sets of time windows with similar
#  brain connectivity patterns, the modular organization of this similarity matrix,
# Scs, is analyzed with the modularity algorithm

script_path = "modul_"+group+".m"
# Start the MATLAB subprocess
matlab_process = subprocess.Popen(['matlab', '-nodisplay', '-nosplash', '-nodesktop', '-r', f"run('{script_path}')"])

# sleep 1 min to make time for the matlab script to be run
time.sleep(35)

In [51]:
# get states, which are averages of similarity windows

out_path_mod = "./outputs/modularity/"
Ci_all = np.array(pd.read_csv(out_path_mod+"Ci_all_"+group+".csv", header=None))
Q_all = np.array(pd.read_csv(out_path_mod+"Q_all_"+group+".csv", header=None))

count = 0
state_count = 0
for patient in dir_list:

    # max number of modules
    max_Ci = max(Ci_all[:,count]) # count = patient

    for m in (range(1,max_Ci+1)):
        ind = [index for index, value in enumerate(Ci_all[:, count]) if value == m]
        S_sum = np.zeros([100,100])
        for i in ind:
            #S_sum = S_sum + S[i]
            S_sum = S_sum + S_group_windows[count][i]
        ST_m = S_sum/len(ind)

        # save
        savefiles(ST_m, "State", './outputs/connect_state/'+group+'/ST_'+str(state_count), "csv")
        savefiles(ST_m, "State", './outputs/connect_state/figures_'+group+'/ST_'+str(state_count), "png")
        
        state_count = state_count +1
        
    count = count+1